Name - Nikhil Saraswat
Roll - 20CS10039

In [ ]:
!pip3 install pytorch-lightning -q -U

In [32]:
# imoprt libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from sklearn.metrics import classification_report
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [33]:
COARSE = 0
FINE = 1
# hyperparameters
HIDDEN_DIM    = [520, 520]
BATCH_SIZE    = [35, 35]
EMBEDDING_DIM = [520,520]
NUM_EPOCHS    = [10, 10]

In [34]:
# create nested dictionary
fine_to_coarse = {}
# add keys and values to the dictionary
fine_to_coarse['I-MusicalGRP'] = 'Group'
fine_to_coarse['B-MusicalGRP'] = 'Group'
fine_to_coarse['I-PublicCorp'] = 'Group'
fine_to_coarse['B-PublicCorp'] = 'Group'
fine_to_coarse['I-PrivateCorp'] = 'Group'
fine_to_coarse['B-PrivateCorp'] = 'Group'
fine_to_coarse['I-OtherCorp'] = 'Group'
fine_to_coarse['B-OtherCorp'] = 'Group'
fine_to_coarse['I-AerospaceManufacturer'] = 'Group'
fine_to_coarse['B-AerospaceManufacturer'] = 'Group'
fine_to_coarse['I-SportsGRP'] = 'Group'
fine_to_coarse['B-SportsGRP'] = 'Group'
fine_to_coarse['I-CarManufacturer'] = 'Group'
fine_to_coarse['B-CarManufacturer'] = 'Group'
fine_to_coarse['I-TechCorp']='Group'
fine_to_coarse['B-TechCorp']='Group'
fine_to_coarse['I-ORG'] = 'Group'
fine_to_coarse['B-ORG'] = 'Group'


fine_to_coarse['I-Scientist'] = 'Person'
fine_to_coarse['B-Scientist'] = 'Person'
fine_to_coarse['I-Artist'] = 'Person'
fine_to_coarse['B-Artist'] = 'Person'
fine_to_coarse['I-Athlete'] = 'Person'
fine_to_coarse['B-Athlete'] = 'Person'
fine_to_coarse['I-Politician'] = 'Person'
fine_to_coarse['B-Politician'] = 'Person'
fine_to_coarse['I-Cleric'] = 'Person'
fine_to_coarse['B-Cleric'] = 'Person'
fine_to_coarse['I-SportsManager'] = 'Person'
fine_to_coarse['B-SportsManager'] = 'Person'
fine_to_coarse['I-OtherPER'] = 'Person'
fine_to_coarse['B-OtherPER'] = 'Person'

fine_to_coarse['I-Medication/Vaccine']='Medical'
fine_to_coarse['B-Medication/Vaccine']='Medical'
fine_to_coarse['I-MedicalProcedure']='Medical'
fine_to_coarse['B-MedicalProcedure']='Medical'
fine_to_coarse['I-AnatomicalStructure']='Medical'
fine_to_coarse['B-AnatomicalStructure']='Medical'
fine_to_coarse['I-Symptom']='Medical'
fine_to_coarse['B-Symptom']='Medical'
fine_to_coarse['I-Disease']='Medical'
fine_to_coarse['B-Disease']='Medical'

fine_to_coarse['I-Facility']='Location'
fine_to_coarse['B-Facility']='Location'
fine_to_coarse['I-OtherLOC']='Location'
fine_to_coarse['B-OtherLOC']='Location'
fine_to_coarse['I-HumanSettlement']='Location'
fine_to_coarse['B-HumanSettlement']='Location'
fine_to_coarse['I-Station']='Location'
fine_to_coarse['B-Station']='Location'

fine_to_coarse['I-Clothing'] = 'Product'
fine_to_coarse['B-Clothing'] = 'Product'
fine_to_coarse['I-Vehicle'] = 'Product'
fine_to_coarse['B-Vehicle'] = 'Product'
fine_to_coarse['I-Food'] = 'Product'
fine_to_coarse['B-Food'] = 'Product'
fine_to_coarse['I-Drink'] = 'Product'
fine_to_coarse['B-Drink'] = 'Product'
fine_to_coarse['I-OtherPROD'] = 'Product'
fine_to_coarse['B-OtherPROD'] = 'Product'

fine_to_coarse['I-VisualWork'] = 'Creative Works' 
fine_to_coarse['B-VisualWork'] = 'Creative Works'
fine_to_coarse['I-MusicalWork'] = 'Creative Works'
fine_to_coarse['B-MusicalWork'] = 'Creative Works'
fine_to_coarse['I-WrittenWork'] = 'Creative Works'
fine_to_coarse['B-WrittenWork'] = 'Creative Works'
fine_to_coarse['I-ArtWork'] = 'Creative Works'
fine_to_coarse['B-ArtWork'] = 'Creative Works'
fine_to_coarse['I-Software'] = 'Creative Works'
fine_to_coarse['B-Software'] = 'Creative Works'
fine_to_coarse['I-OtherCW'] = 'Creative Works'
fine_to_coarse['B-OtherCW'] = 'Creative Works'


fine_to_coarse['O'] = 'O'

In [35]:
def tokenize_sentence(file_path, model_type):
    """
    Tokenize a sentence into words and tags
    Args:
        file_path: path to the file
        model_type: 0 for coarse, 1 for fine
    Returns:
        sentences: list of sentences, each sentence is a list of [word, tag] pairs
    """
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence = []
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                sentences.append(sentence)
                sentence = []
                continue
            if not line:
                continue
            tokens = line.split(' ')
            if model_type == COARSE:
                sentence.append([tokens[0], fine_to_coarse[tokens[3]]])
            else:
                sentence.append([tokens[0], tokens[3]])
        sentences.append(sentence)
    return sentences

In [ ]:
def preprocess(dataset, SEQ_LEN, tag_to_idx, word_to_idx, train=True):
    """
    Preprocess the dataset
    Args:
        dataset: list of sentences, each sentence is a list of [word, tag] pairs
        SEQ_LEN: max length of a sentence
        tag_to_idx: dictionary mapping tags to indices
        word_to_idx: dictionary mapping words to indices
        train: whether the dataset is for training
    """
    # Extract sentences and tags
    sent = [[token.lower() for token, tag in sentence] for sentence in dataset]
    tags = [[tag for token, tag in sentence] for sentence in dataset]

    for i in range(len(sent)):
        while len(sent[i]) < SEQ_LEN:
            sent[i].append('<PAD>')
            tags[i].append('<PAD>')

        if len(sent[i]) > SEQ_LEN:
            sent[i] = sent[i][:SEQ_LEN]
            tags[i] = tags[i][:SEQ_LEN]
    
    if train:
        for sentence_tags in tags:
            for tag in sentence_tags:
                if tag not in tag_to_idx:
                    tag_to_idx[tag] = len(tag_to_idx)
        
        for sentence in sent:
            for word in sentence:
                if word not in word_to_idx:
                    word_to_idx[word] = len(word_to_idx)

    # Convert words and tags to indices
    X = torch.tensor([[word_to_idx.get(word, 1) for word in sentence] for sentence in sent], dtype=torch.int).type(torch.LongTensor)
    Y = torch.tensor([[tag_to_idx[tag] for tag in sentence] for sentence in tags], dtype=torch.int).type(torch.LongTensor)
    
    return X, Y

In [37]:
class NERModel(pl.LightningModule):
    """NER model"""
    def __init__(self, vocab_size, target_size, embedding_dim, hidden_dim, num_layers):
        """
            Constructor
            Args:
                vocab_size: size of the vocabulary
                target_size: number of tags
                embedding_dim: dimension of the word embeddings
                hidden_dim: dimension of the hidden states
                num_layers: number of layers in the LSTM
        """
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True) # if bidirectional = True, output size is 2 * hidden_dim
        self.fc = nn.Linear(hidden_dim * 2, target_size)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        """
            Forward pass
            Args:
                x: input tensor
            Returns:
                output: output tensor
        """
        embedding = self.embedding(x)
        output, _ = self.lstm(embedding)
        output = self.fc(output)
        return nn.functional.log_softmax(output, dim=2)
    
    def training_step(self, batch, batch_idx):
        """
            Training step
            Args:
                batch: batch of data
                batch_idx: index of the batch
            Returns:
                loss: loss value
        """
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        """
            Validation step
            Args:
                batch: batch of data
                batch_idx: index of the batch
            Returns:
                loss: loss value
        """
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('val_loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        """
            Test step
            Args:
                batch: batch of data
                batch_idx: index of the batch
            Returns:
                loss: loss value
        """
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        """
            Configure optimizers
            Returns:
                optimizer: optimizer
        """
        return optim.Adam(self.parameters())

# English Language Models

In [38]:
test_file_path = "en_test.conll"
dev_file_path = "en_dev.conll"
train_file_path = "en_train.conll"

## Fine Model

In [39]:
def en_model(train_file_path, dev_file_path, test_file_path, model_type=FINE):
    """
        This function trains a model on the English dataset and calculates the precision, recall and F1 score on the test set.
        Args:
            train_file_path (str): Path to the training file
            dev_file_path (str): Path to the development file
            test_file_path (str): Path to the test file
            model_type (str): FINE or COARSE
        Returns:
            None
    """
    
    word_to_idx = {"<PAD>": 0, "<UNK>": 1} 
    tag_to_idx = {"<PAD>": 0}
    SEQ_LEN = 25

    trainX, trainY = preprocess(tokenize_sentence(train_file_path, model_type)[1:], SEQ_LEN, tag_to_idx, word_to_idx)
    devX, devY = preprocess(tokenize_sentence(dev_file_path, model_type)[1:], SEQ_LEN, tag_to_idx, word_to_idx, train=False)
    testX, testY = preprocess(tokenize_sentence(test_file_path, model_type)[1:],SEQ_LEN, tag_to_idx, word_to_idx, train=False)

    # print shapes
    print(trainX.shape, trainY.shape)
    print(devX.shape, devY.shape)
    print(testX.shape, testY.shape)

    train_dataset = TensorDataset(trainX, trainY)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE[model_type], shuffle=True, num_workers=2)
    val_dataset = TensorDataset(devX, devY)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE[model_type], shuffle=False, num_workers=2)
    test_dataset = TensorDataset(testX, testY)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE[model_type], shuffle=False, num_workers=2)

    model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
    early_stop_callback = EarlyStopping(monitor='val_loss', patience=5, mode='min')
    trainer = pl.Trainer(max_epochs=NUM_EPOCHS[model_type], callbacks=[early_stop_callback], accelerator='gpu', devices=1)
    trainer.fit(model, train_loader, val_loader)
    trainer.test(dataloaders=test_loader)
    
    # save parameters
    if model_type == COARSE:
        print('english_model_coarse.pt created!')
        torch.save(model.state_dict(), 'english_model_coarse.pt')
    else:
        print('english_model_fine.pt created!')
        torch.save(model.state_dict(), 'english_model_fine.pt')

    # Print 1 sentence for debugging
    model.eval()
    idx_to_tag = {v: k for k, v in tag_to_idx.items()}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    y_true = []
    y_pred = []
                    
    with torch.no_grad():
        for x, y in test_loader:
            # Forward pass
            y_hat = model(x)

            # Get back the sentence
            x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

            # Compute the predicted tags
            y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

            # Compute the true tags
            y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
            print("Sentence")
            for i in range(SEQ_LEN):
                print(x_sent[i], end=" ")
            print()
            print("Predicted tags")
            for i in range(SEQ_LEN):
                print(y_pred[i], end=" ")
            print()
            break
    
    
    # Classification Report
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y in test_loader:
            # Forward pass
            y_hat = model(x)

            # Compute the predicted tags
            y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

            # Compute the true tags
            y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

    print(classification_report(y_true, y_pred))
    
en_model(train_file_path, dev_file_path, test_file_path, model_type=FINE)

torch.Size([16778, 25]) torch.Size([16778, 25])
torch.Size([871, 25]) torch.Size([871, 25])
torch.Size([249980, 25]) torch.Size([249980, 25])


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 17.7 M
1 | lstm      | LSTM             | 4.3 M 
2 | fc        | Linear           | 70.8 K
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
22.1 M    Trainable params
0         Non-trainable params
22.1 M    Total params
88.456    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_2/checkpoints/epoch=7-step=3840.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_2/checkpoints/epoch=7-step=3840.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     0.572156548500061     │
└───────────────────────────┴───────────────────────────┘

english_model_fine.pt created!
Sentence
the species was described by <UNK> <UNK> after the <UNK> t. f. <UNK> . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O O O O B-Politician I-OtherPER O O O I-OtherPER I-Politician I-Politician O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
                         precision    recall  f1-score   support

                  <PAD>       1.00      1.00      1.00   2499089
B-AerospaceManufacturer       0.58      0.45      0.51      1013
  B-AnatomicalStructure       0.51      0.27      0.36      5824
              B-ArtWork       0.33      0.21      0.26      1264
               B-Artist       0.39      0.70      0.50     56981
              B-Athlete       0.39      0.59      0.47     27554
      B-CarManufacturer       0.60      0.31      0.41      2977
               B-Cleric       0.62      0.20      0.30      4725
             B-Clothing       0.48      0.11      0.18      2229
          

## Coarse Model

In [40]:
en_model(train_file_path, dev_file_path, test_file_path, model_type=COARSE)

torch.Size([16778, 25]) torch.Size([16778, 25])
torch.Size([871, 25]) torch.Size([871, 25])
torch.Size([249980, 25]) torch.Size([249980, 25])


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 17.7 M
1 | lstm      | LSTM             | 4.3 M 
2 | fc        | Linear           | 8.3 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
22.1 M    Trainable params
0         Non-trainable params
22.1 M    Total params
88.207    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_3/checkpoints/epoch=6-step=3360.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_3/checkpoints/epoch=6-step=3360.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.3304329216480255     │
└───────────────────────────┴───────────────────────────┘

english_model_coarse.pt created!
Sentence
the species was described by <UNK> <UNK> after the <UNK> t. f. <UNK> . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O O O O Person Person O O Person Person Person Person O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
                precision    recall  f1-score   support

         <PAD>       1.00      1.00      1.00   2499089
Creative Works       0.75      0.51      0.60    169176
         Group       0.72      0.52      0.60    133847
      Location       0.72      0.71      0.72    130520
       Medical       0.68      0.31      0.42     33019
             O       0.94      0.97      0.95   2948139
        Person       0.74      0.89      0.81    290844
       Product       0.48      0.24      0.32     44866

      accuracy                           0.94   6249500
     macro avg       0.75      0.64      0.68   6249500
  weighted avg       0.94      0.94      0.94   6249500



Function to predict NERs for an English Sentence

In [55]:
def predict_NERs_English(sentence, model_type=FINE):
    """
        This function predicts the NERs for a given sentence in English.
        Args:
            sentence (str): The sentence to predict the NERs for
            model_type (str): FINE or COARSE
        Returns:
            None
    """
    # Get the word to index mapping
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    tag_to_idx = {"<PAD>": 0}
    preprocess(tokenize_sentence(train_file_path, model_type)[1:], 25, tag_to_idx, word_to_idx)
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

    # Tokenize the sentence
    sentence = sentence.split()
    sentence = [word.lower() for word in sentence]
    sentence = [word if word in word_to_idx else "<UNK>" for word in sentence]
    tokens = sentence
    sentence = [word_to_idx[word] for word in sentence]
    # Pad the sentence
    if len(sentence) < 25:
        sentence += [0] * (25 - len(sentence))
        tokens += ["<PAD>"] * (25 - len(tokens))
    else:
        sentence = sentence[:25]
        tokens = tokens[:25]
    
    # Convert to tensor
    sentence = torch.tensor(sentence).unsqueeze(0)
    if model_type == COARSE:
        model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
        model.load_state_dict(torch.load('english_model_coarse.pt'))
    else:
        model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
        model.load_state_dict(torch.load('english_model_fine.pt'))
    model.eval()   
    # Predict the tags
    with torch.no_grad():
        y_hat = model(sentence)
        y_pred = [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]
        print("Sentence")
        for i in range(25):
            # print the word and if it is a new word, print the tag
            print(tokens[i], end=" ")
        print()
        print("Predicted tags")
        for i in range(25):
            print(y_pred[i], end=" ")
        print()
predict_NERs_English("I am a student at the University of Toronto", model_type=COARSE)


Sentence
i am a student at the university of toronto <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O Creative Works O O O O O Group Group <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 


# Hindi Language Models

In [56]:
test_file_path = "hi_test.conll"
dev_file_path = "hi_dev.conll"
train_file_path = "hi_train.conll"

## Fine Model

In [57]:
def hi_model(train_file_path, dev_file_path, test_file_path, model_type=FINE):
    """
        This function trains the model for Hindi and prints the classification report.
        Args:
            train_file_path (str): The path to the train file
            dev_file_path (str): The path to the dev file
            test_file_path (str): The path to the test file
            model_type (str): FINE or COARSE
        Returns:
            None
    """
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    tag_to_idx = {"<PAD>": 0}
    SEQ_LEN = 25

    trainX, trainY = preprocess(tokenize_sentence(train_file_path, model_type)[1:], SEQ_LEN, tag_to_idx, word_to_idx)
    devX, devY = preprocess(tokenize_sentence(dev_file_path, model_type)[1:], SEQ_LEN, tag_to_idx, word_to_idx, train=False)
    testX, testY = preprocess(tokenize_sentence(test_file_path, model_type)[1:],SEQ_LEN, tag_to_idx, word_to_idx, train=False)

    # print shapes
    print(trainX.shape, trainY.shape)
    print(devX.shape, devY.shape)
    print(testX.shape, testY.shape)

    train_dataset = TensorDataset(trainX, trainY)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE[model_type], shuffle=True, num_workers=2)
    val_dataset = TensorDataset(devX, devY)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE[model_type], shuffle=False, num_workers=2)
    test_dataset = TensorDataset(testX, testY)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE[model_type], shuffle=False, num_workers=2)

    model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
    early_stop_callback = EarlyStopping(monitor='val_loss', patience=5, mode='min')
    trainer = pl.Trainer(max_epochs=NUM_EPOCHS[model_type], callbacks=[early_stop_callback], accelerator='gpu', devices=1)
    trainer.fit(model, train_loader, val_loader)
    trainer.test(dataloaders=test_loader)
    
    # save parameters
    if model_type == COARSE:
        torch.save(model.state_dict(), 'hindi_model_coarse.pt')
    else:
        torch.save(model.state_dict(), 'hindi_model_fine.pt')

    # Print 1 sentence for debugging
    model.eval()
    idx_to_tag = {v: k for k, v in tag_to_idx.items()}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    y_true = []
    y_pred = []
                    
    with torch.no_grad():
        for x, y in test_loader:
            # Move the data to the device
            # Forward pass
            y_hat = model(x)

            # Get back the sentence
            x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

            # Compute the predicted tags
            y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

            # Compute the true tags
            y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
            print("Sentence")
            for i in range(SEQ_LEN):
                print(x_sent[i], end=" ")
            print()
            print("Predicted tags")
            for i in range(SEQ_LEN):
                print(y_pred[i], end=" ")
            print()
            break
    
    
    # Classification Report
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y in test_loader:
            # Forward pass
            y_hat = model(x)

            # Compute the predicted tags
            y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

            # Compute the true tags
            y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

    print(classification_report(y_true, y_pred))
hi_model(train_file_path, dev_file_path, test_file_path, model_type=FINE)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 8.9 M 
1 | lstm      | LSTM             | 4.3 M 
2 | fc        | Linear           | 70.8 K
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
13.3 M    Trainable params
0         Non-trainable params
13.3 M    Total params
53.363    Total estimated model params size (MB)


torch.Size([9632, 25]) torch.Size([9632, 25])
torch.Size([514, 25]) torch.Size([514, 25])
torch.Size([18399, 25]) torch.Size([18399, 25])


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_4/checkpoints/epoch=8-step=2484.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_4/checkpoints/epoch=8-step=2484.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.2737773656845093     │
└───────────────────────────┴───────────────────────────┘

Sentence
उनकी विशेषताओं आंदोलनों और खेल शैली के कारण उनकी तुलना <UNK> <UNK> से की गई है। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O O O O O O O O O O O O O O O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
                         precision    recall  f1-score   support

                  <PAD>       1.00      1.00      1.00    172076
B-AerospaceManufacturer       0.57      0.05      0.09        85
  B-AnatomicalStructure       0.88      0.65      0.75       485
              B-ArtWork       0.20      0.00      0.01       425
               B-Artist       0.69      0.46      0.55      1847
              B-Athlete       0.61      0.70      0.65      1166
      B-CarManufacturer       0.89      0.90      0.90       146
               B-Cleric       0.61      0.88      0.72       188
             B-Clothing       0.82      0.83      0.82        75
              B-Disease       0.93      0.64      0.76       628
                B-Drink       0.76    

## Coarse Model

In [58]:
hi_model(train_file_path, dev_file_path, test_file_path, model_type=COARSE)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 8.9 M 
1 | lstm      | LSTM             | 4.3 M 
2 | fc        | Linear           | 8.3 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
13.3 M    Trainable params
0         Non-trainable params
13.3 M    Total params
53.113    Total estimated model params size (MB)


torch.Size([9632, 25]) torch.Size([9632, 25])
torch.Size([514, 25]) torch.Size([514, 25])
torch.Size([18399, 25]) torch.Size([18399, 25])


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_5/checkpoints/epoch=6-step=1932.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_5/checkpoints/epoch=6-step=1932.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.19572003185749054    │
└───────────────────────────┴───────────────────────────┘

Sentence
उनकी विशेषताओं आंदोलनों और खेल शैली के कारण उनकी तुलना <UNK> <UNK> से की गई है। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O O O O O O O O O Person Person O O O O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
                precision    recall  f1-score   support

         <PAD>       1.00      1.00      1.00    172076
Creative Works       0.89      0.52      0.66      6689
         Group       0.95      0.78      0.86      9441
      Location       0.91      0.66      0.77     10394
       Medical       0.90      0.71      0.79      2747
             O       0.96      0.99      0.97    244083
        Person       0.74      0.80      0.77     12214
       Product       0.82      0.58      0.68      2331

      accuracy                           0.97    459975
     macro avg       0.90      0.76      0.81    459975
  weighted avg       0.97      0.97      0.96    459975



Function to predict NERs for a Hindi Sentence

In [60]:
def predict_NERs_Hindi(sentence, model_type=FINE):
    # Get the word to index mapping
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    tag_to_idx = {"<PAD>": 0}
    preprocess(tokenize_sentence(train_file_path, model_type)[1:], 25, tag_to_idx, word_to_idx)
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

    # Tokenize the sentence
    sentence = sentence.split()
    sentence = [word.lower() for word in sentence]
    sentence = [word if word in word_to_idx else "<UNK>" for word in sentence]
    tokens = sentence
    sentence = [word_to_idx[word] for word in sentence]
    # Pad the sentence
    if len(sentence) < 25:
        sentence += [0] * (25 - len(sentence))
        tokens += ["<PAD>"] * (25 - len(tokens))
    else:
        sentence = sentence[:25]
        tokens = tokens[:25]
    
    # Convert to tensor
    sentence = torch.tensor(sentence).unsqueeze(0)
    if model_type == COARSE:
        model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
        model.load_state_dict(torch.load('hindi_model_coarse.pt'))
    else:
        model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
        model.load_state_dict(torch.load('hindi_model_fine.pt'))
    model.eval()   
    # Predict the tags
    with torch.no_grad():
        y_hat = model(sentence)
        y_pred = [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]
        print("Sentence")
        for i in range(25):
            # print the word and if it is a new word, print the tag
            print(tokens[i], end=" ")
        print()
        print("Predicted tags")
        for i in range(25):
            print(y_pred[i], end=" ")
        print()
predict_NERs_Hindi("वह २००० के आसपास एक बार भारत में आया था।", model_type=COARSE)



Sentence
वह २००० के आसपास एक बार भारत में आया था। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O O O O O Location O O O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 


# Bengali Language Models

In [61]:
test_file_path = "bn_test.conll"
dev_file_path = "bn_dev.conll"
train_file_path = "bn_train.conll"

## Fine Model

In [62]:
def bn_model(train_file_path, dev_file_path, test_file_path, model_type=FINE):
    """
        This function trains the model for Bengali dataset and prints the classification report
        Args:
            train_file_path: path to the train file
            dev_file_path: path to the dev file
            test_file_path: path to the test file
            model_type: FINE or COARSE

        Returns:
            None
    """
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    tag_to_idx = {"<PAD>": 0}
    SEQ_LEN = 25

    trainX, trainY = preprocess(tokenize_sentence(train_file_path, model_type)[1:], SEQ_LEN, tag_to_idx, word_to_idx)
    devX, devY = preprocess(tokenize_sentence(dev_file_path, model_type)[1:], SEQ_LEN, tag_to_idx, word_to_idx, train=False)
    testX, testY = preprocess(tokenize_sentence(test_file_path, model_type)[1:],SEQ_LEN, tag_to_idx, word_to_idx, train=False)

    # print shapes
    print(trainX.shape, trainY.shape)
    print(devX.shape, devY.shape)
    print(testX.shape, testY.shape)

    train_dataset = TensorDataset(trainX, trainY)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE[model_type], shuffle=True, num_workers=2)
    val_dataset = TensorDataset(devX, devY)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE[model_type], shuffle=False, num_workers=2)
    test_dataset = TensorDataset(testX, testY)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE[model_type], shuffle=False, num_workers=2)

    model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
    early_stop_callback = EarlyStopping(monitor='val_loss', patience=5, mode='min')
    trainer = pl.Trainer(max_epochs=NUM_EPOCHS[model_type], callbacks=[early_stop_callback], accelerator='gpu', devices=1)
    trainer.fit(model, train_loader, val_loader)
    trainer.test(dataloaders=test_loader)
    
    # save parameters
    if model_type == COARSE:
        torch.save(model.state_dict(), 'bengali_model_coarse.pt')
    else:
        torch.save(model.state_dict(), 'bengali_model_fine.pt')

    # Print 1 sentence for debugging
    model.eval()
    idx_to_tag = {v: k for k, v in tag_to_idx.items()}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    y_true = []
    y_pred = []
                    
    with torch.no_grad():
        for x, y in test_loader:

            # Forward pass
            y_hat = model(x)

            # Get back the sentence
            x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

            # Compute the predicted tags
            y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

            # Compute the true tags
            y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
            print("Sentence")
            for i in range(SEQ_LEN):
                print(x_sent[i], end=" ")
            print()
            print("Predicted tags")
            for i in range(SEQ_LEN):
                print(y_pred[i], end=" ")
            print()
            break
    
    
    # Classification Report
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y in test_loader:
            # Forward pass
            y_hat = model(x)

            # Compute the predicted tags
            y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

            # Compute the true tags
            y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

    print(classification_report(y_true, y_pred))
bn_model(train_file_path, dev_file_path, test_file_path, model_type=FINE)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


torch.Size([9708, 25]) torch.Size([9708, 25])
torch.Size([507, 25]) torch.Size([507, 25])
torch.Size([19859, 25]) torch.Size([19859, 25])


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 11.6 M
1 | lstm      | LSTM             | 4.3 M 
2 | fc        | Linear           | 70.8 K
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
16.0 M    Trainable params
0         Non-trainable params
16.0 M    Total params
63.931    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_6/checkpoints/epoch=7-step=2224.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_6/checkpoints/epoch=7-step=2224.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.27344316244125366    │
└───────────────────────────┴───────────────────────────┘

Sentence
প্রোপেলারটি একটি ডি <UNK> এয়ারক্রাফ্ট কোম্পানি স্থির পিচ টাইপ ছিল। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O O B-OtherPROD O B-ORG O O O O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
                         precision    recall  f1-score   support

                  <PAD>       1.00      1.00      1.00    240692
B-AerospaceManufacturer       0.29      0.05      0.09        97
  B-AnatomicalStructure       0.91      0.63      0.75       532
              B-ArtWork       0.22      0.02      0.03       455
               B-Artist       0.60      0.47      0.53      2744
              B-Athlete       0.51      0.38      0.44      1086
      B-CarManufacturer       0.75      0.93      0.83        84
               B-Cleric       0.66      0.65      0.65       240
             B-Clothing       0.30      0.71      0.42        17
              B-Disease       0.92      0

## Coarse Model

In [63]:
bn_model(train_file_path, dev_file_path, test_file_path, model_type=COARSE)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


torch.Size([9708, 25]) torch.Size([9708, 25])
torch.Size([507, 25]) torch.Size([507, 25])
torch.Size([19859, 25]) torch.Size([19859, 25])


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 11.6 M
1 | lstm      | LSTM             | 4.3 M 
2 | fc        | Linear           | 8.3 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
15.9 M    Trainable params
0         Non-trainable params
15.9 M    Total params
63.681    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_7/checkpoints/epoch=7-step=2224.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_7/checkpoints/epoch=7-step=2224.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.1752036213874817     │
└───────────────────────────┴───────────────────────────┘

Sentence
প্রোপেলারটি একটি ডি <UNK> এয়ারক্রাফ্ট কোম্পানি স্থির পিচ টাইপ ছিল। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
Predicted tags
O O Group Group O Group O O O O <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
                precision    recall  f1-score   support

         <PAD>       1.00      1.00      1.00    240692
Creative Works       0.78      0.61      0.68      8334
         Group       0.66      0.84      0.74      8621
      Location       0.84      0.77      0.80     10677
       Medical       0.86      0.69      0.76      2585
             O       0.96      0.98      0.97    208686
        Person       0.88      0.70      0.78     14628
       Product       0.58      0.57      0.58      2252

      accuracy                           0.96    496475
     macro avg       0.82      0.77      0.79    496475
  weighted avg       0.96      0.96      0.96    496475



Function to predict NERs for a Bengali Sentence

In [66]:
def predict_NERs_Bengali(sentence, model_type=FINE):
    """
        This function predicts the NERs for a given sentence
        Args:
            sentence: the sentence in Bengali for which NERs are to be predicted
            model_type: FINE or COARSE

        Returns:
            None
    """
    # Get the word to index mapping
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    tag_to_idx = {"<PAD>": 0}
    preprocess(tokenize_sentence(train_file_path, model_type)[1:], 25, tag_to_idx, word_to_idx)
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

    # Tokenize the sentence
    sentence = sentence.split()
    sentence = [word.lower() for word in sentence]
    sentence = [word if word in word_to_idx else "<UNK>" for word in sentence]
    tokens = sentence
    sentence = [word_to_idx[word] for word in sentence]
    # Pad the sentence
    if len(sentence) < 25:
        sentence += [0] * (25 - len(sentence))
        tokens += ["<PAD>"] * (25 - len(tokens))
    else:
        sentence = sentence[:25]
        tokens = tokens[:25]
    
    # Convert to tensor
    sentence = torch.tensor(sentence).unsqueeze(0)
    if model_type == COARSE:
        model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
        model.load_state_dict(torch.load('bengali_model_coarse.pt'))
    else:
        model = NERModel(len(word_to_idx), len(tag_to_idx), EMBEDDING_DIM[model_type], HIDDEN_DIM[model_type], 1)
        model.load_state_dict(torch.load('bengali_model_fine.pt'))
    model.eval()   
    # Predict the tags
    with torch.no_grad():
        y_hat = model(sentence)
        y_pred = [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]
        print("Sentence")
        for i in range(25):
            # print the word and if it is a new word, print the tag
            print(tokens[i], end=" ")
        print()
        print("Predicted tags")
        for i in range(25):
            print(y_pred[i], end=" ")
        print()
predict_NERs_Bengali("অ্যাপল একটি আইটি কোম্পানি যা ম্যাকবুক, ট্যাবলেট এবং অন্যান্য ডিজিটাল ডিভাইস তৈরি করে। অ্যাপল একটি আইটি কোম্পানি যা ম্যাকবুক,ট্যাবলেট এবং অন্যান্য ডিজিটাল ডিভাইস তৈরি করে। ", model_type=FINE)



Sentence
অ্যাপল একটি <UNK> কোম্পানি যা <UNK> ট্যাবলেট এবং অন্যান্য ডিজিটাল ডিভাইস তৈরি করে। অ্যাপল একটি <UNK> কোম্পানি যা <UNK> এবং অন্যান্য ডিজিটাল ডিভাইস তৈরি করে। 
Predicted tags
B-PublicCorp O B-OtherPROD I-OtherPROD O B-OtherPROD O O O O O O O O O B-OtherPROD I-OtherPROD O O O O O O O O 
